# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [5]:
#Your code here
import pandas as pd

# Load the dataset
df = pd.read_csv('homepage_actions.csv')

# Display the first few rows of the dataset
print(df.head())

# Check the dimensions of the dataset
print("Dimensions:", df.shape)

# Count the number of unique viewers and clickers
num_viewers = df['id'].nunique()
num_clickers = df[df['action'] == 'click']['id'].nunique()

print("Number of viewers:", num_viewers)
print("Number of clickers:", num_clickers)

# Check for anomalies: anyone clicked without viewing
clicked_without_viewing = df[df['action'] == 'click']['id'].isin(df[df['action'] == 'view']['id'])
anomalies = clicked_without_viewing.sum()

print("Number of anomalies (clicked without viewing):", anomalies)

# Check for overlap between control and experiment groups
control_group = df[df['group'] == 'control']['id']
experiment_group = df[df['group'] == 'experiment']['id']

overlap = control_group.isin(experiment_group).sum()
print("Number of IDs in both control and experiment groups:", overlap)


                    timestamp      id       group action
0  2016-09-24 17:42:27.839496  804196  experiment   view
1  2016-09-24 19:19:03.542569  434745  experiment   view
2  2016-09-24 19:36:00.944135  507599  experiment   view
3  2016-09-24 19:59:02.646620  671993     control   view
4  2016-09-24 20:26:14.466886  536734  experiment   view
Dimensions: (8188, 4)
Number of viewers: 6328
Number of clickers: 1860
Number of anomalies (clicked without viewing): 1860
Number of IDs in both control and experiment groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [6]:
#Your code here
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest

# Load the dataset
df = pd.read_csv('homepage_actions.csv')

# Count the number of viewers and clickers in control and experiment groups
control_viewers = df[(df['group'] == 'control') & (df['action'] == 'view')]['id'].nunique()
control_clickers = df[(df['group'] == 'control') & (df['action'] == 'click')]['id'].nunique()

experiment_viewers = df[(df['group'] == 'experiment') & (df['action'] == 'view')]['id'].nunique()
experiment_clickers = df[(df['group'] == 'experiment') & (df['action'] == 'click')]['id'].nunique()

# Perform the two-sample proportion test
count = [control_clickers, experiment_clickers]
nobs = [control_viewers, experiment_viewers]

stat, pval = proportions_ztest(count, nobs)

# Print the test statistic and p-value
print("Test Statistic:", stat)
print("p-value:", pval)

Test Statistic: -2.618563885349469
p-value: 0.008830075576595804


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [7]:
#Your code here
import pandas as pd

# Load the dataset
df = pd.read_csv('homepage_actions.csv')

# Calculate the click-through rate of the control group
control_viewers = df[(df['group'] == 'control') & (df['action'] == 'view')]['id'].nunique()
control_clickers = df[(df['group'] == 'control') & (df['action'] == 'click')]['id'].nunique()
control_ctr = control_clickers / control_viewers

# Calculate the expected number of clicks for the experiment group
experiment_viewers = df[(df['group'] == 'experiment') & (df['action'] == 'view')]['id'].nunique()
expected_experiment_clicks = control_ctr * experiment_viewers

print("Expected number of clicks for the experiment group:", expected_experiment_clicks)


Expected number of clicks for the experiment group: 838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [10]:
#Your code here
import pandas as pd
import math

# Load the dataset
df = pd.read_csv('homepage_actions.csv')

# Calculate the expected number of clicks for the experiment group (from previous step)
expected_experiment_clicks = 838.0168067226891

# Calculate the actual number of clicks for the experiment group
experiment_clickers = df[(df['group'] == 'experiment') & (df['action'] == 'click')]['id'].nunique()

# Calculate the standard deviation for the binomial distribution
experiment_viewers = df[(df['group'] == 'experiment') & (df['action'] == 'view')]['id'].nunique()
p = expected_experiment_clicks / experiment_viewers  # Estimated click-through rate
variance = experiment_viewers * p * (1 - p)
std_dev = math.sqrt(variance)

# Calculate the number of standard deviations
num_std_deviations = (experiment_clickers - expected_experiment_clicks) / std_dev

print("Number of standard deviations:", num_std_deviations)


Number of standard deviations: 3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [11]:
#Your code here
import scipy.stats as stats

# Calculate the p-value using the normal distribution
p_value = 1 - stats.norm.cdf(3.6625360854823588)

print("p-value:", p_value)


p-value: 0.00012486528006949715


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **The analysis involving the calculation of the expected number of clicks for the experiment group, the number of standard deviations, and the p-value using the normal distribution provides a different perspective on verifying the results of the previous statistical test.**

By comparing the obtained p-value from the normal distribution approach with the p-value from the previous statistical test (such as the two-sample proportion test), we can assess whether the results are consistent or not. If the p-values from both approaches are similar, it suggests that the experimental homepage was indeed more effective than the control group.

It's important to note that both approaches make certain assumptions. The previous statistical test assumed that the click-through rates in the control and experiment groups followed a binomial distribution. The analysis involving the normal distribution assumes that the click-through rates are approximately normally distributed.

If the results from the two approaches are consistent and indicate that the experimental homepage was more effective than the control group, it strengthens the evidence in favor of the experimental changes made to the homepage. However, if the results differ significantly, it may require further investigation or consideration of additional factors to understand the inconsistency.

Ultimately, it is important to interpret the results of both analyses in the context of the specific experiment and the goals of the website A/B testing.**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.